<a href="https://colab.research.google.com/github/sureshkumar64/Mlprojects/blob/main/EV_Charging_Station_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()

Saving ev-charging-stations-india.csv to ev-charging-stations-india (1).csv


Upload widget is only available when the cell has been executed in the current browser session. Please rerun this cell to enable.
Saving ev-charging-stations-india.csv to ev-charging-stations-india.csv

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings



print("--- EV Charging Station Recommender System ---")

# Step 1: Load and clean the dataset

file_path = 'ev-charging-stations-india.csv'

try:
    df = pd.read_csv(file_path, low_memory=False)
    print(f"Dataset '{file_path}' loaded successfully.")
    print(f"Initial dataset shape: {df.shape}")
except Exception as e:
    print(f"Error loading the dataset from '{file_path}': {e}")
    print("Please ensure the 'ev-charging-stations-india.csv' file is correctly uploaded and accessible.")
    exit() # Exit if the file cannot be loaded



# Select relevant columns for recommendation.
df_clean = df[['name', 'address', 'city', 'state']].copy()

# Handle missing values by filling with empty strings for text columns
for col in ['name', 'address', 'city', 'state']:
    df_clean[col] = df_clean[col].fillna('')

# Create a combined 'features' column for vectorization

df_clean['combined_features'] = df_clean['name'] + ' ' + \
                                df_clean['address'] + ' ' + \
                                df_clean['city'] + ' ' + \
                                df_clean['state']


print("\nData cleaning and 'combined_features' creation complete.")
print(df_clean.head())

# Step 2: Simulate user preferences

user_preferences = {
    'fast_charger': True,
    'connector_types': ['CHAdeMO', 'CCS', 'Type 2'],
    'preferred_city': 'Delhi',
    'min_power_kw': 50
}

# Translate user preferences into a text string that can be vectorized
user_preference_string = ""
if user_preferences['fast_charger']:
    user_preference_string += " fast charger DCFC CHAdeMO CCS "
if user_preferences['connector_types']:
    user_preference_string += " ".join(user_preferences['connector_types']) + " "
if user_preferences['preferred_city']:
    user_preference_string += user_preferences['preferred_city'] + " "
if user_preferences['min_power_kw']:


    if user_preferences['min_power_kw'] >= 50:
        user_preference_string += " 50kW 100kW high-power "

print(f"\nSimulated User Preference String: '{user_preference_string.strip()}'")

# Step 3: Use cosine similarity to match preferences

vectorizer = TfidfVectorizer(stop_words='english', min_df=1)

# Fit the vectorizer on all combined features (stations and user preference)
all_features_text = df_clean['combined_features'].tolist() + [user_preference_string]
tfidf_matrix = vectorizer.fit_transform(all_features_text)

# The last vector in tfidf_matrix corresponds to the user's preferences
user_vector = tfidf_matrix[-1]

# Compute cosine similarity between the user preference vector and all station feature vectors
station_vectors = tfidf_matrix[:-1]
cosine_similarities = cosine_similarity(user_vector, station_vectors).flatten()

# Add similarity scores to the DataFrame
df_clean['similarity_score'] = cosine_similarities

print("\nCosine similarity calculation complete.")

# Step 4: Display recommended stations
n_recommendations = 10 # Number of top recommendations to display

# Sort stations by similarity score in descending order
recommended_stations = df_clean.sort_values(by='similarity_score', ascending=False)


recommended_stations = recommended_stations[recommended_stations['similarity_score'] > 0]

print(f"\n--- Top {n_recommendations} Recommended EV Charging Stations for the User ---")
if recommended_stations.empty:
    print("No recommendations found based on the provided preferences. Try adjusting user preferences or dataset features.")
else:
    # Display relevant columns for recommendations - use available columns
    display_cols = ['name', 'address', 'city', 'state', 'similarity_score']
    print(recommended_stations[display_cols].head(n_recommendations).to_string())

print("\n--- Recommender System Process Complete ---")

--- EV Charging Station Recommender System ---
Dataset 'ev-charging-stations-india.csv' loaded successfully.
Initial dataset shape: (1547, 7)

Data cleaning and 'combined_features' creation complete.
                                                name  \
0                 Neelkanth Star DC Charging Station   
1                       Galleria DC Charging Station   
2  Highway Xpress (Jaipur-Delhi) DC charging station   
3                  Food Carnival DC Charging Station   
4                  Food Carnival AC Charging Station   

                                             address      city          state  \
0  Neelkanth Star Karnal, NH 44, Gharunda, Kutail...  Gurugram        Haryana   
1  DLF Phase IV, Sector 28, Gurugram, Haryana 122022  Gurugram        Haryana   
2  Jaipur to Delhi Road, Behror Midway, Behror, R...    Behror      Rajasthan   
3  Fun and Food Carnival, NH 58, Khatauli Bypass,...  Khatauli  Uttar Pradesh   
4  NH 58, Khatauli Bypass, Bhainsi, Uttar Pradesh...  Khat

--- EV Charging Station Recommender System ---
Dataset 'ev-charging-stations-india.csv' loaded successfully.
Initial dataset shape: (1547, 7)

Data cleaning and 'combined_features' creation complete.
                                                name  \
0                 Neelkanth Star DC Charging Station   
1                       Galleria DC Charging Station   
2  Highway Xpress (Jaipur-Delhi) DC charging station   
3                  Food Carnival DC Charging Station   
4                  Food Carnival AC Charging Station   

                                             address      city          state  \
0  Neelkanth Star Karnal, NH 44, Gharunda, Kutail...  Gurugram        Haryana   
1  DLF Phase IV, Sector 28, Gurugram, Haryana 122022  Gurugram        Haryana   
2  Jaipur to Delhi Road, Behror Midway, Behror, R...    Behror      Rajasthan   
3  Fun and Food Carnival, NH 58, Khatauli Bypass,...  Khatauli  Uttar Pradesh   
4  NH 58, Khatauli Bypass, Bhainsi, Uttar Pradesh...  Khatauli  Uttar Pradesh   

                                   combined_features  
0  Neelkanth Star DC Charging Station Neelkanth S...  
1  Galleria DC Charging Station DLF Phase IV, Sec...  
2  Highway Xpress (Jaipur-Delhi) DC charging stat...  
3  Food Carnival DC Charging Station Fun and Food...  
4  Food Carnival AC Charging Station NH 58, Khata...  

Simulated User Preference String: 'fast charger DCFC CHAdeMO CCS CHAdeMO CCS Type 2 Delhi  50kW 100kW high-power'

Cosine similarity calculation complete.

--- Top 10 Recommended EV Charging Stations for the User ---
                                          name                                                                                                       address       city        state  similarity_score
1336                   EESL High court station                                                                       High Court Metro Station Chennai (CMRL)    Chennai   Tamil Nadu          0.093143
1337                   EESL High court station                                                                       High Court Metro Station Chennai (CMRL)    Chennai   Tamil Nadu          0.093143
830                    EESL High court station                                                                       High Court Metro Station Chennai (CMRL)    Chennai   Tamil Nadu          0.093143
829                    EESL High court station                                                                       High Court Metro Station Chennai (CMRL)    Chennai   Tamil Nadu          0.093143
770                               EESL Block B                                                          Shop No. B-29, Inner Circle, New Delhi, Delhi 110001  New Delhi        Delhi          0.063507
1518                               Tata Power   Naniwala Bagh Commercial Complex, Azadpur, Commercial Complex, Gopal Nagar, Azadpur, New Delhi, Delhi 110009      Delhi        Delhi          0.063201
863                      MG DELHI SHIVAJI MARG                                                                PLOT NO 13 SHIVAJI MARG DELHI 110015 NEW DELHI  NEW DELHI        Delhi          0.062143
87    Hotel Daulat Nikol Fast Charging Station                        Hotel Daulat Food Mall, Mumbai Agra Highway, Shirwade (Vani) Tal: Niphad, Dist: Nashik     Nashik  Maharashtra          0.061538
958                      MG DELHI SHIVAJI MARG                                                                PLOT NO 13 SHIVAJI MARG DELHI 110015 NEW DELHI  NEW DELHI    Delhi NCR          0.061203
766                            EESL Snow White                                                        D-8, D Block, Connaught Place, New Delhi, Delhi 110001  New Delhi        Delhi          0.060917

--- Recommender System Process Complete ---